In [1]:
import os
import random
import traceback
from dotenv import load_dotenv
from pymilvus import connections, utility, Collection, FieldSchema, DataType, CollectionSchema
from sentence_transformers import SentenceTransformer
from pymilvus import MilvusClient
from openai import OpenAI

load_dotenv()
openai_client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

MILVUS_HOST = os.getenv("MILVUS_HOST")
MILVUS_PORT = os.getenv("MILVUS_PORT")

DB_NAME = os.getenv("DATABASE_NAME")
COLLECTION_NAME = os.getenv("COLLECTION_NAME")

CACHE_DB_NAME = os.getenv("CACHE_DATABASE_NAME")
CACHE_COLLECTION_NAME=os.getenv("CACHE_COLLECTION_NAME")

DIMENSION = 512  # SentenceTransformer all-mpnet-base-v2 的輸出維度

In [2]:
def insert_question_into_cache(cache_host_name, cache_port, cache_db_name, cache_collection_name, question, answer):
    #print(f"insert_question_into_cache: {cache_host_name}/{cache_port}/{cache_db_name}/{cache_collection_name}/{question}/{answer}")
    embedder = SentenceTransformer('distiluse-base-multilingual-cased-v1')
    client = None
    try:
        client = MilvusClient(
            uri=f"http://{cache_host_name}:{cache_port}",
            db_name=cache_db_name
        )
        entities = []
        embedding = embedder.encode(question)
        entities.append({"embedding": embedding, "text": answer}) # 同時儲存 embedding 和 answer
        client.insert(collection_name=cache_collection_name, data=entities)
    except Exception as e:
        traceback.print_exc()
    finally:
        if client:
            client.close()
def query_cache(cache_host_name, cache_port, cache_db_name, cache_collection_name, question):
    #print(f"query_cache: {cache_host_name}/{cache_port}/{cache_db_name}/{cache_collection_name}/{question}")
    embedder = SentenceTransformer('distiluse-base-multilingual-cased-v1')
    threshold=0.75
    xq = embedder.encode(question)
    ## IP: inner product
    ##search_params = {"metric_type": "IP", "param": {"nprobe": 1,"range_filter": threshold}} # I don't know why range_filter does not work well here.
    search_params = {"metric_type": "IP", "param": {"nprobe": 1,"radius": threshold}} # I don't know why radius does not work well here
    client = None
    try:
        client = MilvusClient(
            uri=f"http://{cache_host_name}:{cache_port}",
            db_name=cache_db_name
        )
        res=client.search(
                collection_name=cache_collection_name,
                anns_field="embedding",
                data=[xq],
                limit=1,
                search_params=search_params,
                output_fields=["id","text"]
                )
        
        if res and res[0] and res[0][0] and res[0][0]["distance"] > threshold and res[0][0]["entity"]: ## because radius does not work well
        ##if res and res[0] and res[0][0] and res[0][0]["entity"]: 
            print("cache hitted!!")
            #print(res)
            resp=(res[0][0]["entity"]["text"])
            return resp
        else:
            print("cache missed!!")
            return None
    except Exception as e:
        traceback.print_exc()
        return None
    finally:
        if client:
            client.close()

In [3]:
def query_and_response_with_cache(host_name, port, db_name, collection_name, user_query, cache_host_name, 
                                  cache_port, cache_db_name, cache_collection_name):
    print(f"query_and_response_with_cache: {host_name}/{port}/{db_name}/{collection_name}/{user_query}")
    resp=query_cache(cache_host_name, cache_port, cache_db_name, cache_collection_name, user_query) 
    if resp:
        return resp
    embedder = SentenceTransformer('distiluse-base-multilingual-cased-v1')
    xq = embedder.encode(user_query)
    search_params = {"metric_type": "COSINE", "param": {"nprobe": 16}}

    client = None
    try:
        client = MilvusClient(
            uri=f"http://{host_name}:{port}",
            db_name=db_name
        )
        res=client.search(
                collection_name=collection_name,
                anns_field="embedding",
                data=[xq],
                limit=1,
                search_params=search_params,
                output_fields=["id","text"]
                )
        
        if res and res[0] and res[0][0] and res[0][0]["entity"]:
            #data_id=(res[0][0]["id"])
            #raw_response_material_data=client.get(collection_name=collection_name, ids=[data_id], output_fields=["text"])
            #res[0][0]["entity"]["text"]
            #print(res[0][0]["entity"]["text"])
            prompt = f"""使用以下上下文回答問題：
            {res[0][0]["entity"]["text"]}
    
            問題：{user_query}
            """
            response = openai_client.chat.completions.create(model="gpt-4o-mini", 
            messages=[{"role": "user", "content": prompt}], temperature=0)
            insert_question_into_cache(cache_host_name, cache_port, cache_db_name, cache_collection_name, user_query, 
                                       response.choices[0].message.content.strip())
            return response.choices[0].message.content.strip()
        else:
            return "找不到相關資訊"
    except Exception as e:
        traceback.print_exc()
        return "發生錯誤"
    finally:
        if client:
            client.close()

In [4]:
%%time
result=query_and_response_with_cache(host_name=MILVUS_HOST, 
                                     port=MILVUS_PORT,
                                     db_name=DB_NAME, 
                                     collection_name=COLLECTION_NAME, 
                                     user_query="管理委員會之職務是什麼",
                                     cache_host_name=MILVUS_HOST, 
                                     cache_port=MILVUS_PORT, 
                                     cache_db_name=CACHE_DB_NAME, 
                                     cache_collection_name=CACHE_COLLECTION_NAME)
print(result)

query_and_response_with_cache: 127.0.0.1/19530/law_database/law_text_embeddings/管理委員會之職務是什麼
cache missed!!
管理委員會的職務包括以下幾項：

1. 執行區分所有權人會議的決議事項。
2. 負責共有及共用部分的清潔、維護、修繕及一般改良。
3. 確保公寓大廈及其周圍的安全及環境維護。
4. 提出住戶共同事務的改進建議。
5. 制止住戶的違規行為並提供相關資料。
6. 協調住戶違反相關規定的情況。
7. 管理收益、公共基金及其他經費的收支、保管及運用。
8. 保管規約、會議紀錄、使用執照謄本、竣工圖說等相關文件。
9. 委任、僱傭及監督管理服務人。
10. 提出及公告會計報告、結算報告及其他管理事項。
11. 點收及保管共用部分及其附屬設施設備。
12. 申報公共安全檢查與消防安全設備檢修，並執行改善措施。
13. 處理其他依本條例或規約所定的事項。
CPU times: user 1.86 s, sys: 320 ms, total: 2.18 s
Wall time: 14.2 s


In [5]:
%%time
result=query_and_response_with_cache(host_name=MILVUS_HOST, 
                                     port=MILVUS_PORT,
                                     db_name=DB_NAME, 
                                     collection_name=COLLECTION_NAME, 
                                     user_query="管理委員會職務內容包含什麼",
                                     cache_host_name=MILVUS_HOST, 
                                     cache_port=MILVUS_PORT, 
                                     cache_db_name=CACHE_DB_NAME, 
                                     cache_collection_name=CACHE_COLLECTION_NAME)
print(result)

query_and_response_with_cache: 127.0.0.1/19530/law_database/law_text_embeddings/管理委員會職務內容包含什麼
cache hitted!!
管理委員會的職務包括以下幾項：

1. 執行區分所有權人會議的決議事項。
2. 負責共有及共用部分的清潔、維護、修繕及一般改良。
3. 確保公寓大廈及其周圍的安全及環境維護。
4. 提出住戶共同事務的改進建議。
5. 制止住戶的違規行為並提供相關資料。
6. 協調住戶違反相關規定的情況。
7. 管理收益、公共基金及其他經費的收支、保管及運用。
8. 保管規約、會議紀錄、使用執照謄本、竣工圖說等相關文件。
9. 委任、僱傭及監督管理服務人。
10. 提出及公告會計報告、結算報告及其他管理事項。
11. 點收及保管共用部分及其附屬設施設備。
12. 申報公共安全檢查與消防安全設備檢修，並執行改善措施。
13. 處理其他依本條例或規約所定的事項。
CPU times: user 550 ms, sys: 72.2 ms, total: 622 ms
Wall time: 2.72 s


In [6]:
%%time
result=query_and_response_with_cache(host_name=MILVUS_HOST, 
                                     port=MILVUS_PORT,
                                     db_name=DB_NAME, 
                                     collection_name=COLLECTION_NAME, 
                                     user_query="本條例用辭定義",
                                     cache_host_name=MILVUS_HOST, 
                                     cache_port=MILVUS_PORT, 
                                     cache_db_name=CACHE_DB_NAME, 
                                     cache_collection_name=CACHE_COLLECTION_NAME)
print(result)

query_and_response_with_cache: 127.0.0.1/19530/law_database/law_text_embeddings/本條例用辭定義
cache missed!!
本條例用辭定義了與公寓大廈相關的多個重要概念，包括：

1. **公寓大廈**：指具有明確界線的建築物及其基地，能夠區分為數個部分。
2. **區分所有**：指多個人對一建築物的專有部分擁有所有權，並對共用部分按比例擁有權利。
3. **專有部分**：公寓大廈中具有獨立使用性的部分，屬於區分所有的標的。
4. **共用部分**：不屬於專有部分的其他部分及附屬建築物，供所有住戶共同使用。
5. **約定專用部分**：共用部分經約定供特定區分所有權人使用的部分。
6. **約定共用部分**：專有部分經約定供所有住戶共同使用的部分。
7. **區分所有權人會議**：全體區分所有權人召開的會議，討論共同事務及權利義務相關事項。
8. **住戶**：包括區分所有權人、承租人及經區分所有權人同意的使用者。
9. **管理委員會**：由區分所有權人選任的組織，負責執行會議決議及管理維護工作。
10. **管理負責人**：未成立管理委員會時，由區分所有權人推選的負責人。
11. **管理服務人**：由管理委員會或管理負責人聘用的管理維護人員或公司。
12. **規約**：區分所有權人為增進共同利益而制定的共同遵守事項。

這些定義有助於明確公寓大廈的管理及使用規範。
CPU times: user 1.68 s, sys: 119 ms, total: 1.8 s
Wall time: 18.4 s


-----

# References
- https://www.restack.io/p/gptcache-knowledge-gptcache-milvus-cat-ai
- https://community.aws/content/2juMSXyaSX2qelT4YSdHBrW2D6s/bridging-the-efficiency-gap-mastering-llm-caching-for-next-generation-ai-part-2
- https://github.com/zilliztech/GPTCache
- https://milvus.io/docs
- https://www.sbert.net/docs/sentence_transformer/pretrained_models.html